In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv'

In [3]:
df = pd.read_csv(url)
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


**!!!** **Sorun1:** Ay ve Gün numeric değil object tipindedir. Bu durum sadece sayıları anlayan **Doğrusal Regresyon** modelimiz için bir sorundur.

* Gün sütununu ele alacak olursak 7 kategoriden oluştuğunu söyleyebiliriz. Spesifik olarak, bu örnekte haftanın her günü için bir sütun oluşturulacaktır. Yani bir sütun yediye ayrıştırılacak. 

Makine öğreniminde veri işleme adımında, bir makine öğrenimi modelini beslemeden önce genellikle verilerimizi belirli şekillerde hazırlamamız gerekir. Örneklerden biri, kategorik veriler üzerinde One-Hot kodlama yapmaktır.


One-Hot Kodlama, veri işlemede kategorik verilere uygulanan ve bunları makine öğrenimi algoritmalarında kullanılmak üzere ikili vektör gösterimine dönüştüren bir süreçtir.

> **ONE-HOT ENCODER (Scikit-Learn)** , olası her değer için bir sütun oluşturur ve uygun sütuna bir koyar. Kategoriler arasında sıralama olmadığında kullanılan yöntemdir. Cinsiyet değişkenini bu yöntemle dönüştürdüğümüzde kadın ve erkek için iki ayrı sütun oluşturur ve 0-1 olarak değer atar. 

## Veri Temizleme

> **GET DUMMIES (Pandas)**
One Hot Encoder yöntemi ile aynıdır yalnızca dönüşüm yaparken farklı kütüphaneleri kullanırlar. Bu yöntemde değişkenin tüm kategorileri veri setine sütun olarak eklenir ve her satır için sahip olunan kategori sütunda 1 diğer kategoriler ise 0 olarak etiketlenir.

NOT!! Çok kategoriye sahip değişkenler için bu dönüşüm çok boyutluluk ve sparse veri problemine neden olabilir. 

In [4]:
df = pd.get_dummies(df)

In [5]:
df.columns # verisetinin sütunlarına bakıyoruz.

Index(['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain',
       'area', 'month_apr', 'month_aug', 'month_dec', 'month_feb', 'month_jan',
       'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov',
       'month_oct', 'month_sep', 'day_fri', 'day_mon', 'day_sat', 'day_sun',
       'day_thu', 'day_tue', 'day_wed'],
      dtype='object')

Bu noktada kolonlar da bir artış olduğunu gözlemleyebiliyoruz. Gün ve Ay kolonlarına uyguladığımız kategorik değişken dönüşümü ile her farklı veri için yeni bir kolon oluşturmuş olduk. 

Gün kolonunu ele alalım. Pazartesi den Pazar gününe kadar 7 niteliğimiz var. İlk 6 nitelik 1 veya 0 olduğunda 7. nitelik otomatikmen 1 veya 0 olmuş oluyor. Bu nedenle 7. niteliğin kolonunu bırakmamız herhangi bir sorun teşkil etmeyip aksine verisetinin sadeliği ve anlaşılırlığı için önemlidir. Bu nedenle her one-hot kodlanmış özelliğin bir sütununu bırakmalıyız. 

Bu durumda hangisini bıraktığımızın öneminin olmadığı Ay ve Gün kolonlarında türeyen 1 er kolonu bırakalım.

In [6]:
df.drop(labels=["month_dec","day_sat"], axis=1, inplace=True)

In [7]:
df.max() - df.min()  # Her kolon için değer aralıklarını gözlemleyelim.

X               8.00
Y               7.00
FFMC           77.50
DMC           290.20
DC            852.70
ISI            56.10
temp           31.10
RH             85.00
wind            9.00
rain            6.40
area         1090.84
month_apr       1.00
month_aug       1.00
month_feb       1.00
month_jan       1.00
month_jul       1.00
month_jun       1.00
month_mar       1.00
month_may       1.00
month_nov       1.00
month_oct       1.00
month_sep       1.00
day_fri         1.00
day_mon         1.00
day_sun         1.00
day_thu         1.00
day_tue         1.00
day_wed         1.00
dtype: float64

Değişkenlerin bazıları, DMC ve DC gibi nispeten yüksek varyansa sahipken, diğerleri haftanın günü gibi 0 ile 1 arasında sınırlandırılmıştır.

## Model Eğitimi

**BİLGİ:** 
**Basit doğrusal regresyon;** bağımsız değişken (X)ile bağımlı değişken (Y)deki değişimi açıklamayı, bağımsız değişkendeki bir birimlik değişimin bağımlı değişken üzerindeki etkisini ölçmeyi amaçlar.

Temel amaç, bağımlı ve bağımsız değişken arasındaki ilişkiyi ifade eden doğrusal fonksiyonu bulmaktır.

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
data = df.copy()

In [10]:
target = data.pop("area")  
# "area" kolonunu datadan çıkardık aynı zamanda pop fonksiyonu ile çıkarttığımız area kolonunu target(hedef) değişkenine attık.

In [11]:
target.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: area, dtype: float64

In [12]:
data.head() 

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,month_may,month_nov,month_oct,month_sep,day_fri,day_mon,day_sun,day_thu,day_tue,day_wed
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,1,0,0,0,0,0,1,0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,1,0,0,0,0,0,0,0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,0,0,1,0,0,0


**Regresyonu modelleyebilmek için bir model nesnesi oluşturalım. Bunun için scikit-learn kütüphanesinden yararlanalım.**

In [13]:
lr = LinearRegression(fit_intercept=True)

# fit_intercept=False, çizgi y kesme noktası orijine (0, 0) zorlanır.
# fit_intercept=True, y kesme noktası en uygun çizgi tarafından belirlenir.

**Modelimizi fit() metoduyla kuralım. Doğrusal bir fit en küçük kareler yöntemiyle bulunur.**

*Vikipedia:* En küçük kareler yöntemi, birbirine bağlı olarak değişen iki fiziksel büyüklük arasındaki matematiksel bağlantıyı, mümkün olduğunca gerçeğe uygun bir denklem olarak yazmak için kullanılan, standart bir regresyon yöntemidir. Bir başka deyişle **bu yöntem, ölçüm sonucu elde edilmiş veri noktalarına "mümkün olduğu kadar yakın" geçecek bir fonksiyon eğrisi bulmaya yarar.** Gauss-Markov Teoremi'ne göre en küçük kareler yöntemi, regresyon için optimal yöntemdir.

In [14]:
lr.fit(data, target)

LinearRegression()

In [15]:
lr.intercept_   # β0 katsayısını çağırmak için

51.62412253340599

In [16]:
lr.coef_        # β1 katsayısını çağırmak için

array([  2.25583295,  -0.14765158,  -0.09527277,   0.20106326,
        -0.12880031,  -0.54416333,   1.29619892,  -0.13476015,
         1.97427446,  -2.81544852, -47.3782107 ,  -0.4961627 ,
       -41.78836131, -32.60912303, -18.49932411, -40.66272755,
       -51.60076681, -34.58175393, -51.78831038,  21.59715384,
        26.3537085 , -19.40992987, -13.44064682, -14.26533092,
        -9.73800989, -11.61710499, -13.93079221])

**!!!** Scikit-learn kütüphanesinde eğitim verisinden elde edilen parametrelerin sonuna alt tire(_) konur. Böylece bulunan parametreler ile kullanıcıdan alınan parametreler birbirinden ayrılır.

**Modelimizin nasıl performans gösterdiğini görmek için R ^ 2 ve Root Mean Squared Error(RMSE)’a bakalım.**

In [17]:
lr.score(data, target)  # R2 değeri

0.04578209650808518

R2 modelin açıklanabilirliği ile ilgili bir değerdir. **R2 elimizdeki bağımsız değişkenleri kullandığımızda bağımlı değişkendeki değişimin yüzde kaçını açıklayabiliyoruz** bilgisini bize sunar. 

In [18]:
from sklearn.metrics import mean_squared_error

In [19]:
predictions = lr.predict(data) # tahmin değerlerini oluşturduk

In [20]:
mse = mean_squared_error(target, predictions) # Ortalama kare hata
mse

3859.0724526251074

* **Ortalama kare hata**(mean_squared_error) bir regresyon eğrisinin bir dizi noktaya ne kadar yakın olduğunu söyler. MSE, bir makine öğrenmesi modelinin, tahminleyicinin performansını ölçer, **her zaman pozitif değerlidir ve MSE değeri sıfıra yakın olan tahminleyicilerin daha iyi bir performans gösterdiği söylenebilir.**

In [59]:
rmse = np.sqrt(mse) # Kök Ortalama Kare Hata
rmse

62.12143311792724

* Bir makine öğrenmesi modelinin, tahminleyicinin tahmin ettiği değerler ile gerçek değerleri arasındaki uzaklığın bulunmasında sıklıkla kullanılan, hatanın büyüklüğünü ölçen kuadratik bir metriktir. RMSE tahmin hatalarının (kalıntıların) standart sapmasıdır. Başka bir deyişle, verilere en iyi uyan çizgi etrafında o verilerin ne kadar yoğun olduğunu söyler. RMSE değeri 0’dan ∞’a kadar değişebilir. **Negatif yönelimli puanlar yani daha düşük değerlere sahip tahminleyiciler daha iyi performans gösterir. RMSE değerinin sıfır olması modelin hiç hata yapmadığı anlamına gelir.**

##### Elde Ettiğimiz Son İki Değeri Yorumlayalım
Değerlere göre modelimizin kötü olduğunu söyleyebiliriz. 

1’e yaklaşan bir R ^ 2’ye ya da 0’a yakın bir RMSE’ye sahip olabilirdik. 

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                    shuffle=True,
                                                    test_size=0.5,
                                                    random_state=49)

* **shuffle :** Bölmeden önce verilerin karıştırılıp karıştırılmayacağını belirler.

* **random_state :** Bölmeyi uygulamadan önce verilere uygulanan karıştırmayı kontrol eder.

* **test_size :** Float ise, 0.0 ile 1.0 arasında olmalı ve test bölmesine dahil edilecek veri kümesinin oranını temsil etmelidir.

#### Eğitim Verilerine Uygun Model

In [62]:
lr_split = LinearRegression(fit_intercept=True)
lr_split.fit(X_train,y_train)

LinearRegression()

In [63]:
def calc_ISE(X_train, y_train, model):
    predictions = model.predict(X_train)
    mse = mean_squared_error(y_train, predictions)
    rmse = np.sqrt(mse)
    return model.score(X_train, y_train), rmse

In [64]:
def calc_OSE(X_test, y_test, model):
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    return model.score(X_test, y_test), rmse

In [65]:
is_r2, ise = calc_ISE(X_train,y_train, lr_split)

os_r2, ose = calc_OSE(X_test, y_test, lr_split)

# Veri kümesi boyutlarını göster.
data_list = (("R^2_in", is_r2), 
             ("R^2_out", os_r2), 
             ("ISE", ise), 
             ("OSE", ose))

for item in data_list:
    print("{:10}: {}".format(item[0],item[1]))

R^2_in    : 0.07513974961762804
R^2_out   : -0.005167798223800313
ISE       : 24.75445904929311
OSE       : 86.03512342431831


**Yorumlama**
Örnek içi R ^ 2’nin oldukça düşük olduğunu görebiliriz ancak burada ilginç olan, örnek dışı R ^ 2’nin daha düşük olmasıdır. Aslında, biraz sıfırın altında. Daha da fazlasını söylemek RMSE değerleridir. Modelin gördüğü veriler için RMSE (ISE veya eğitim hatası), modelin hiç görmediği veriler için RMSE’den (3 faktörü) önemli ölçüde daha düşüktür (OSE veya test hatası). Makine öğreniminde konuşmamız, modelimiz fazla yorucudur, yani gördüğü veriler üzerinde çok daha iyi bir iş çıkarır (yani, eğitimli model iyi genelleşmez). Egzersiz hatası ile test hatası arasındaki boşluk ne kadar büyükse, aşırı uyarlama da o kadar fazla olur. Donanımı donatarak verileri ezberleyebilirsiniz. Bir arama tablosu oluşturma gibi daha fazla olur.

Bu yüzden buradaki en büyük paket, modelinizin nasıl çalıştığına dair doğru bir görüntü elde etmek için eğitim hatasını ve test hatasını hesaplamanız gerektiğidir. Bu, başlangıçta bazı verileri tutmayı gerektirir, böylece modelinizi hiç görmediği verilerde test edebilirsiniz. Sklearn’ün train_test_split’sinde bunu yaptığınızı gösterdim.

### Eğitim / Test Bölünmesinin Dezavantajı

##### Çoklu Eğitim / Test Bölmeleri

In [66]:
# Random_state değerleri dizisi yarat.
random_states = np.random.randint(1,100,size=5)
random_states

array([40,  2, 29, 96, 59])

In [67]:
for random_state in random_states:
    # Verileri rastgele duruma göre böl.
    X_train, X_test, y_train, y_test = train_test_split(data, target,
                                                        shuffle = True,
                                                        test_size = 0.5,
                                                        random_state = random_state)
    
    # Örnek model
    lr = LinearRegression(fit_intercept=True)

    # Eğitim modeli
    lr.fit(X_train,y_train)

    # Önemli metrikleri yakalayın.
    is_r2, ise = calc_ISE(X_train, y_train, lr)
    os_r2, ose = calc_OSE(X_test, y_test, lr)
    
    # Yuvarlak değerler
    is_r2, os_r2 = round(is_r2, 4), round(os_r2, 4)
    ise, ose = round(ise, 4), round(ose, 4)
    
    # Anahtar metriklerini yazdır.
    print('Random State: {}'.format(random_state))
    print('IS_R^2: {} | IS_RMSE: {}'.format(is_r2, ise))
    print('OS_R^2: {} | OS_RMSE: {}'.format(os_r2, ose))
    print('-'*34)

Random State: 40
IS_R^2: 0.0934 | IS_RMSE: 27.3559
OS_R^2: -0.0028 | OS_RMSE: 85.0894
----------------------------------
Random State: 2
IS_R^2: 0.0814 | IS_RMSE: 68.3748
OS_R^2: -0.1342 | OS_RMSE: 58.3456
----------------------------------
Random State: 29
IS_R^2: 0.0722 | IS_RMSE: 83.5326
OS_R^2: -0.8835 | OS_RMSE: 31.7918
----------------------------------
Random State: 96
IS_R^2: 0.0647 | IS_RMSE: 70.2541
OS_R^2: -0.0577 | OS_RMSE: 54.575
----------------------------------
Random State: 59
IS_R^2: 0.0938 | IS_RMSE: 47.5284
OS_R^2: -0.0389 | OS_RMSE: 76.1462
----------------------------------


**Çıkarımlar**
* R ^ 2, örneklemenin tersine her zaman örneklemde daha yüksektir.
* RMSE, örnekten örneklere kıyasla büyük değişkenlik göstermektedir.

**Tartışma**
Örnek olarak R ^ 2’nin daha yüksek olması sürpriz değildir. Buradaki sürpriz RMSE. Özellikle ilginç olan, bazen eğitim hatasının test hatasından daha yüksek olduğu ve bazen bunun tersi olmasıdır. Bu küçük bir veri kümesidir, bu nedenle hedef değişkendeki çarpık dağılım bu etkiyi yaratmaktadır. Çok daha büyük bir veri kümesi daha küçük bir dereceye kadar etkilenecektir.